In [1]:
import pandas as pd
from data_processor import DataProcessor as dp

In [2]:
fatture_categorizzate = pd.read_csv('categorized_output.csv')
print(f"Fatture categorizzate dataset shape: {fatture_categorizzate.shape}")
print(fatture_categorizzate.sample(20).to_string(index=False))

Fatture categorizzate dataset shape: (25194, 16)
                      SourceName  CODICE_COMMITTENTE  ALIQUOTA_IVA       DATA       P_IVA UNITAMISURA  IMPORTO_TOTALE_DOCUMENTO     NUMERO TIPO_DOCUMENTO                                  RAGIONE_SOCIALE  SEDE_INDIRIZZO_CESSIONARIO  QUANTITA                                                                                                                                                                                             DESCRIZIONE  IMPONIBILE_IMPORTO  PREZZO_UNITARIO            CATEGORIA
             IT04157540966_4iikp        1.009275e+10          10.0 2024-03-29  2827030962         NaN                   1263.29      4iikp           TD01                              METRO Italia S.p.A.     VIA FRANCESCO ALGAROTTI       1.0                                                                                                                                                                                  G.500 LIEVITO  IN PANI            

In [ ]:
clustered_fatture = dp.sequential_cluster(fatture_categorizzate, threshold=0.8)
reduced_row = dp.representatives(clustered_fatture)


Initial dataframe shape:
(25194, 16)
Dataframe shape after clustering:
(25194, 17)
Dataframe shape after selecting representatives:
(3988, 17)
Reduced dataset shape: (3988, 17)
